<a href="https://colab.research.google.com/github/aaalexlit/omdena_climate_change_challenge_notebooks/blob/main/Inference_using_multivers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

## Clone multivers repo

In [1]:
%%capture
!git clone https://github.com/dwadden/multivers.git

## Setup virtual environment and requirements
virtual environment is needed to avoid conflicts with the packages already installed on Colab

In [3]:
%%capture
!pip install virtualenv
!virtualenv multivers
!source /content/multivers/bin/activate; pip install -r /content/multivers/requirements.txt

## Download the model checkpoints

In [4]:
!python /content/multivers/script/get_checkpoint.py longformer_large_science
!python /content/multivers/script/get_checkpoint.py scifact


--2023-02-25 06:39:29--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/longformer_large_science.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.218.221.137, 3.5.78.146, 3.5.78.210, ...
Connecting to scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)|52.218.221.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1646843693 (1.5G) [binary/octet-stream]
Saving to: ‘checkpoints/longformer_large_science.ckpt’

checkpoints/longfor 100%[===================>]   1.53G  21.3MB/s    in 2m 54s  

2023-02-25 06:42:23 (9.02 MB/s) - ‘checkpoints/longformer_large_science.ckpt’ saved [1646843693/1646843693]

--2023-02-25 06:42:24--  https://scifact.s3.us-west-2.amazonaws.com/longchecker/latest/checkpoints/scifact.ckpt
Resolving scifact.s3.us-west-2.amazonaws.com (scifact.s3.us-west-2.amazonaws.com)... 52.218.176.233, 52.218.216.49, 3.5.78.146, ...
Connecting to scifact.s3.us-west-2.am

And move the models to the folder where multivers is looking for them

In [5]:
!mv /content/checkpoints /content/multivers/checkpoints

## If CPU is not available only 
delete line `    model.to(f"cuda:{args.device}")` from `predict.py` otherwise it would throw an error `RuntimeError: No CUDA GPUs are available
`

In [6]:
import torch
import os
if not torch.cuda.is_available():
  ret_code = os.system("sed -i '40d' /content/multivers/multivers/predict.py")
  print(f"exit code = {ret_code}")

# First inference attempt

In [2]:
!gdown https://drive.google.com/uc?id=1KFKGaaiKwx81B_hIeSyvyqiMPDy0e6VZ -O corpus.jsonl
!gdown https://drive.google.com/uc?id=1QwqYmNWQhlsy4Wabg5Lqtv0H1wDjW2N2 -O claims.jsonl

Downloading...
From: https://drive.google.com/uc?id=1KFKGaaiKwx81B_hIeSyvyqiMPDy0e6VZ
To: /content/corpus.jsonl
100% 87.7k/87.7k [00:00<00:00, 79.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1QwqYmNWQhlsy4Wabg5Lqtv0H1wDjW2N2
To: /content/claims.jsonl
100% 629/629 [00:00<00:00, 1.42MB/s]


In [7]:
from timeit import default_timer as timer
start = timer()

In [8]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=100 \
        --input_file=/content/claims.jsonl \
        --corpus_file=/content/corpus.jsonl \
        --output_file=/content/preds.jsonl

Downloading: 100% 803/803 [00:00<00:00, 1.02MB/s]
Downloading: 100% 1.74G/1.74G [00:56<00:00, 31.0MB/s]
Downloading: 100% 899k/899k [00:00<00:00, 6.37MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 3.48MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 6.62MB/s]
100% 1/1 [00:15<00:00, 15.25s/it]


In [9]:
end = timer()
print(f"Took {(end - start):.0f} seconds")

Took 120 seconds


In [10]:
!cat /content/preds.jsonl

{"id": 0, "evidence": {}}
{"id": 1, "evidence": {"31": {"label": "CONTRADICT", "sentences": []}, "32": {"label": "CONTRADICT", "sentences": [1]}}}
{"id": 2, "evidence": {}}


## The result from the inference above visualized
```
{"id": 1, "claim": "Arctic sea ice has expanded in recent years."}
```
the evidence from the below abstracts contradict with the claim
```
{"doc_id": 31, "title": "An Updated Assessment of the Changing Arctic Sea Ice Cover", "abstract": ["Sea ice is an essential component of the Arctic climate system.", "The sea cover has undergone substantial changes in past 40+ years, including decline areal extent all months (strongest during summer), thinning, loss multiyear cover, earlier melt onset and retreat, later freeze-up advance.", "In 10 these trends have been further reinforced, though (not statistically significant at p &lt;0.05) some parameters (e.g., extent) over decade are more moderate.", "Since 2011, observing capabilities improved significantly, collection first basin-wide routine observations freeboard thickness by radar laser altimeters (except summer).", "addition, data from a year-long field campaign 2019–2020 promises to yield bounty situ that will vastly improve understanding small-scale processes interactions between ice, ocean, atmosphere, as well provide valuable validation for satellite missions.", "impacts within clear already affecting humans flora fauna.", "Impacts outside Arctic, while garnering much attention, remain unclear.", "future dependent on CO2 emissions, but seasonally ice-free Ocean likely coming decades.", "However, year-to-year variability causes considerable uncertainty exactly when this happen.", "also challenge seasonal prediction."]}
{"doc_id": 32, "title": "Arctic sea ice in transformation: A review of recent observed changes and impacts on biology and human activity", "abstract": ["Sea ice in the Arctic is one of most rapidly changing components global climate system.", "Over past few decades, summer areal extent has declined over 30, and all months show statistically significant declining trends.", "New satellite missions techniques have greatly expanded information on sea thickness, but many uncertainties remain data long-term records are sparse.", "However, thickness observations other satellite-derived indicate a 40 decline due large part to loss thicker, older cover.", "The changes happening faster than models projected.", "With continued increasing temperatures, ice-free conditions likely sometime coming though there substantial exact timing high interannual variability will as decreases.", "already having an impact flora fauna Arctic.", "Some species face challenges future, while new habitat open up for species.", "also affecting peoples living working Native communities facing their traditional ways life, opportunities shipping, fishing, natural resource extraction."]}
```

For the rest of the claims there's NOT ENOUGH INFO

# Testing inference for Claims extracted from this article  
https://www.nationalgeographic.com/environment/article/amazon-rainforest-now-appears-to-be-contributing-to-climate-change

There are 38 claims (they're not necessarily "good" claims, claim extraction is also AI assisted) and 30 abstracts for each claim

In [11]:
!gdown https://drive.google.com/uc?id=16FmVPslnSDYBevOms4C_AJTCZi9TC1UK -O corpus_1.jsonl
!gdown https://drive.google.com/uc?id=1xUA2faMjS4-9ahZjovMnOIlTWu8-DeYX -O claims_1.jsonl

Downloading...
From: https://drive.google.com/uc?id=16FmVPslnSDYBevOms4C_AJTCZi9TC1UK
To: /content/corpus_1.jsonl
100% 1.32M/1.32M [00:00<00:00, 131MB/s]
Downloading...
From: https://drive.google.com/uc?id=1xUA2faMjS4-9ahZjovMnOIlTWu8-DeYX
To: /content/claims_1.jsonl
100% 13.0k/13.0k [00:00<00:00, 22.7MB/s]


In [16]:
start = timer()

In [17]:
!source /content/multivers/bin/activate; python /content/multivers/multivers/predict.py \
        --checkpoint_path=/content/multivers/checkpoints/scifact.ckpt \
        --batch_size=10 \
        --input_file=/content/claims_1.jsonl \
        --corpus_file=/content/corpus_1.jsonl \
        --output_file=/content/preds_1.jsonl

100% 117/117 [03:29<00:00,  1.79s/it]


In [18]:
!cat /content/preds_1.jsonl

{"id": 0, "evidence": {}}
{"id": 1, "evidence": {}}
{"id": 2, "evidence": {}}
{"id": 3, "evidence": {}}
{"id": 4, "evidence": {}}
{"id": 5, "evidence": {}}
{"id": 6, "evidence": {}}
{"id": 7, "evidence": {}}
{"id": 8, "evidence": {}}
{"id": 9, "evidence": {}}
{"id": 10, "evidence": {}}
{"id": 11, "evidence": {}}
{"id": 12, "evidence": {}}
{"id": 13, "evidence": {}}
{"id": 14, "evidence": {}}
{"id": 15, "evidence": {}}
{"id": 16, "evidence": {}}
{"id": 17, "evidence": {}}
{"id": 18, "evidence": {}}
{"id": 19, "evidence": {}}
{"id": 20, "evidence": {}}
{"id": 21, "evidence": {}}
{"id": 22, "evidence": {"681": {"label": "SUPPORT", "sentences": [4]}, "684": {"label": "SUPPORT", "sentences": [8]}}}
{"id": 23, "evidence": {}}
{"id": 24, "evidence": {}}
{"id": 25, "evidence": {}}
{"id": 26, "evidence": {}}
{"id": 27, "evidence": {}}
{"id": 28, "evidence": {}}
{"id": 29, "evidence": {}}
{"id": 30, "evidence": {}}
{"id": 31, "evidence": {}}
{"id": 32, "evidence": {}}
{"id": 33, "evidence": {}}


In [19]:
end = timer()
print(f"Took {(end - start):.0f} seconds")

Took 274 seconds


## The result from the inference above visualized
```
{"id": 22, "claim": "Droughts come more often and, in some areas, last longer."}
```
the evidence from the below abstracts contradict with the claim
```
{"doc_id": 681, 
"title": "Five Decades of Observed Daily Precipitation Reveal Longer and More Variable Drought Events Across Much of the Western United States", 
"abstract": [
  "Multiple lines of evidence suggest climate change will result in increased precipitation variability and consequently more frequent extreme events.", 
"These hydroclimatic changes likely have significant socioecological impacts, especially across water-limited regions.", 
"Here we present an analysis daily meteorological observations from 1976 to 2019 at 337 long-term weather stations distributed the western United States (US).", 
"In addition widespread warming (0.2 °C ± 0.01°C/decade, maximum temperature), observed trends reduced annual (−2.3 1.5 mm/decade) most region, with increasing interannual precipitation.", 
"Critically, showed that extreme-duration drought became common, increases both mean longest dry interval between events (0.6 0.2, 2.4 0.3 days/decade) greater these intervals.", 
"findings indicate that, against a backdrop drying, large regions US are experiencing intensification variability, detrimental consequences for essential ecosystem services."]}
{"doc_id": 684, 
"title": "National-scale analysis of simulated hydrological droughts (1891–2015)", 
"abstract": [
  "Droughts are phenomena that affect people and ecosystems in a variety of ways.", 
  "One way to help with resilience future droughts is understand the characteristics historic how these have changed over recent past.", 
  "Although, on average, Great Britain experiences relatively wet climate it also prone periods low rainfall which can lead droughts.", 
  "Until recently research into has been neglected compared other natural hazards such as storms floods.", 
  "This study first use national-scale gridded hydrological model characterise across last century.", 
  "Firstly, performance at flows assessed found simulate well many catchments Britain.", 
  "Next, threshold level method applied time series monthly mean river flow soil moisture identify (1891–2015).", 
  "It shown output be used drought periods.", 
  "A quantitative assessment shows groundwater-dependent areas typically experience more severe droughts, longer durations rather than higher intensities.", 
  "There substantial spatial temporal variability characteristics, but there no consistent changes through time."]}
```

For the rest of the claims there's NOT ENOUGH INFO